In [8]:
import pandas as pd
import numpy as np

import ast

import snap

In [52]:
path = 'output/'
cluster_data = pd.read_csv(path+'1_step_user_groups.csv', sep='\t')

In [13]:
cluster_tags = pd.read_csv(path+'1_step_tags_clusters.txt', sep='\t')
cluster_tags.head()

,n_cluster,dim,hashtags
0,-1,1618,/
1,0,6828,/
2,1,16,"interiorinspo,decora,interiordecor,interiorsty..."
3,2,35,"disney,instacat,gato,princess,ladyboss,poetryc..."
4,3,14,"europa,fashionstylist,voyage,argentina,aloha,e..."


In [53]:
# filter clusters that are not relevant
cluster_data = cluster_data[cluster_data['num_users'] > 1]
cluster_data.head()

,id_cluster,num_users,list_users
0,1,17,"('citychic4ever__', 0.9411764705882353), ('raw..."
1,2,16,"('len_cap', 1.0), ('crown_rising', 0.823529411..."
2,3,15,"('rimaglio', 1.0), ('pollyonvoyage', 0.8235294..."
3,4,1,"('ber_maria', 0.8823529411764706)"
4,5,2,"('mynnorthamerica', 0.8823529411764706), ('gra..."


In [11]:
def filterUsers(userlist):
    userlist = list(ast.literal_eval(userlist))
    result = []
    for userdata in userlist:
        if userdata[1] <= 0.5:
            result.append(userdata[0])
    return result

In [14]:
def getCompleteUserlist(userlist):
    userlist = list(ast.literal_eval(userlist))
    result = []
    for userdata in userlist:
        result.append(userdata[0])
    return result

In [54]:
# keep only relevant users (== consumers)
cluster_data['relevant_users'] = cluster_data.apply(lambda x: filterUsers(x['list_users']), axis=1)
cluster_data['all_users'] = cluster_data.apply(lambda x: getCompleteUserlist(x['list_users']), axis=1)
cluster_data.head()

TypeError: ("'float' object has no attribute '__getitem__'", u'occurred at index 3')

In [22]:
# read source network
network = snap.LoadEdgeListNet('data/followers_network.csv', '\t')

V = network.GetNodes()

In [46]:
# use all the users to expand the clusters
temp_result = []
for c in range(1, cluster_data.shape[0]+1):
    c_users = cluster_data[cluster_data['id_cluster'] == c]['all_users']
    
    it = network.BegNI()
    tag_ids = snap.TIntV()
    for i in range(V):
        nid = it.GetId()
        type = network.GetStrAttrDatN(nid, 'type')
        username = network.GetStrAttrDatN(nid, 'content')

        if type == 'user' and username in c_users:
            # add all tags of user
            tags = snap.TIntV()
            snap.GetNodesAtHop(network, nid, 2, tags, True)
            tag_ids.AddV(tags)

        it.Next()
        
    tag_ids.Merge() 
    
    alltags = set()
    for tid in tag_ids:
        tagname = network.GetStrAttrDatN(tid, 'content')
        alltags.add(tagname)
        
    c_tags = set(cluster_tags[cluster_tags['n_cluster'] == c]['hashtags'].values[0].split(','))
    newtags = alltags - c_tags
    
    temp_result.append(tuple((c, newtags)))

In [51]:
c_users

27    [cok_alvarez, medy1978, nathiaroga]
Name: all_users, dtype: object

In [48]:
extended_cluster_tags = pd.DataFrame(temp_result, columns=['n_cluster', 'hashtags'])
extended_cluster_tags

,n_cluster,hashtags
0,1,{}
1,2,{}
2,3,{}
3,4,{}
4,5,{}
5,6,{}
6,7,{}
7,8,{}
8,9,{}
9,10,{}
